In [17]:
import folium
import dask.dataframe as dd
import h3_pandas

In [ ]:
columns = [
    'medallion', 'hack_license', 'vendor_id', 'rate_code', 'store_and_fwd_flag',
    'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs',
    'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'
    ]

# Read the data into a Dask DataFrame
df1 = dd.read_csv('trip_data_1.csv', header=None, names=columns)
df1 = df1.loc[1:]

df2 = dd.read_csv('trip_data_2.csv', header=None, names=columns)
df2 = df2.loc[1:]

df3 = dd.read_csv('trip_data_3.csv', header=None, names=columns)
df3 = df3.loc[1:]

df = dd.concat([df1, df2, df3])

df.astype({'pickup_longitude': 'float64', 'pickup_latitude': 'float64', 'dropoff_longitude':'float64','dropoff_latitude':'float64','rate_code':'int32'}).dtypes 

# Assuming df is your Dask DataFrame
df['pickup_datetime'] = dd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = dd.to_datetime(df['dropoff_datetime'])
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_day_of_week'] = df['pickup_datetime'].dt.dayofweek


# Filter the data
df_filtered = df[(df['hack_license']== 'F49FD0D84449AE7F72F3BC492CD6C754') & (df['pickup_day_of_week'] < 5) & (df['pickup_hour'] > 1) & (df['pickup_hour'] < 5)]

# Convert to Pandas DataFrame
df_filtered = df_filtered.compute()

In [ ]:
# Convert to H3 grid at resolution 11
resolution = 11
df = df.h3.geo_to_h3(resolution)

df = df.reset_index(['h3_11'] )

df['h3_index_count'] = df.groupby(['h3_11'])['h3_11'].transform('count')



In [19]:
# Create a map centered around New York City
m = folium.Map(location=[40.730610, -73.935242], zoom_start=12)

# Add markers for pickup and dropoff locations
for index, row in df_filtered.iterrows():
    pickup_location = (row['pickup_latitude'], row['pickup_longitude'])
    dropoff_location = (row['dropoff_latitude'], row['dropoff_longitude'])
    
    folium.Marker(
        location=pickup_location,
        popup='Pickup',
        icon=folium.Icon(color='green')
    ).add_to(m)
    
    folium.Marker(
        location=dropoff_location,
        popup='Dropoff',
        icon=folium.Icon(color='red')
    ).add_to(m)

# Display the map
# m.save('pickup_dropoff_map_200_points.html')  # Save the map to an HTML file
m